In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow.keras import layers, models


In [2]:

def load_custom_dataset(dataset_path, img_size=(224, 224)):
    images = []
    labels = []

    for folder_name in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder_name)
        if os.path.isdir(folder_path):
            label = int(folder_name)
            
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
               
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, img_size)
                
                images.append(image)
                labels.append(label)  
    images = np.array(images)
    labels = np.array(labels)
    return images, labels


In [3]:

def create_alexnet_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model


In [5]:

# Load data
dataset_path = "C:/Users/gliff/Documents/software/train"
x, y = load_custom_dataset(dataset_path)
#x = x.astype("float32") / 255.0
x = np.expand_dims(x, axis=-1)  # Expand dimension for channels
num_classes = len(np.unique(y))
y = tf.keras.utils.to_categorical(y, num_classes)

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Define input shape
input_shape = x_train.shape[1:]


KeyboardInterrupt: 

In [ ]:

# Create and compile the model
model = create_alexnet_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Train the model
history = model.fit(x_train, y_train, epochs=1, batch_size=32, validation_data=(x_test, y_test))

# Save the model
model.save("my_alexnet_model")
